<a href="https://colab.research.google.com/github/koushika894/Character-Recognization-GPU/blob/main/gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=f7a0fd012778e58cc9f5f90842b72c79778e18c1775d24e472eac8de6b6c3c9d
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import pycuda.gpuarray as gpuarray
from sklearn.metrics import accuracy_score, classification_report
import time

In [ ]:
N_data = np.array([
    [[1, 0, 0, 0, 1],
     [1, 1, 0, 0, 1],
     [1, 0, 1, 0, 1],
     [1, 0, 0, 1, 1],
     [1, 0, 0, 0, 1]],

    [[1, 0, 0, 0, 1],
     [1, 1, 0, 0, 1],
     [1, 0, 1, 0, 1],
     [1, 0, 0, 1, 1],
     [0, 0, 0, 0, 0]],

    [[0, 0, 0, 0, 0],
     [1, 1, 0, 0, 1],
     [1, 0, 1, 0, 1],
     [1, 0, 0, 1, 1],
     [1, 0, 0, 0, 1]],

    [[0, 0, 0, 0, 0],
     [1, 1, 0, 0, 1],
     [1, 0, 1, 0, 1],
     [1, 0, 0, 1, 1],
     [0, 0, 0, 0, 0]],

    [[1, 0, 0, 1, 0],
     [1, 1, 0, 1, 0],
     [1, 0, 1, 1, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]],

    [[0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [1, 0, 1, 0, 0],
     [1, 1, 1, 0, 0],
     [1, 0, 1, 0, 0]],

    [[1, 0, 0, 0, 1],
     [1, 1, 0, 0, 1],
     [1, 0, 1, 0, 1],
     [1, 0, 0, 1, 1],
     [1, 0, 0, 0, 1]],

    [[0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 1, 0, 1, 0],
     [0, 1, 1, 1, 0],
     [0, 1, 0, 1, 0]],

    [[1, 0, 1, 0, 0],
     [1, 1, 1, 0, 0],
     [1, 0, 1, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]],

    [[0, 0, 0, 0, 0],
     [0, 1, 0, 0, 0],
     [0, 0, 1, 0, 1],
     [0, 0, 1, 1, 1],
     [0, 0, 1, 0, 1]]
])


In [ ]:
A_data = np.array([
    [[0, 0, 1, 0, 0],
     [0, 1, 0, 1, 0],
     [0, 1, 1, 1, 0],
     [0, 1, 0, 1, 0],
     [0, 1, 0, 1, 0]],

    [[0, 0, 1, 0, 0],
     [0, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]],

    [[0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 1, 0, 0],
     [0, 1, 1, 1, 0],
     [1, 0, 0, 0, 1]],

    [[0, 1, 0, 0, 0],
     [1, 0, 1, 0, 0],
     [1, 1, 1, 0, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 1, 0, 0]],

    [[0, 0, 0, 1, 0],
     [0, 0, 1, 0, 1],
     [0, 0, 1, 1, 1],
     [0, 0, 1, 0, 1],
     [0, 0, 1, 0, 1]],

    [[0, 0, 1, 1, 0],
     [0, 1, 0, 0, 1],
     [0, 1, 1, 1, 1],
     [0, 1, 0, 0, 1],
     [0, 1, 0, 0, 1]],

    [[0, 1, 1, 0, 0],
     [1, 0, 0, 1, 0],
     [1, 1, 1, 1, 0],
     [1, 0, 0, 1, 0],
     [1, 0, 0, 1, 0]],

    [[0, 1, 0, 0, 0],
     [1, 0, 1, 0, 0],
     [1, 1, 1, 0, 0],
     [1, 0, 1, 0, 0],
     [0, 0, 0, 0, 0]],

    [[0, 0, 0, 1, 0],
     [0, 0, 1, 0, 1],
     [0, 0, 1, 1, 1],
     [0, 0, 1, 0, 1],
     [0, 0, 0, 0, 0]],

    [[0, 0, 0, 0, 0],
     [0, 0, 1, 0, 0],
     [0, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [0, 0, 0, 0, 0]]
])

In [ ]:
R_data = np.array([
    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]],

    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]],

    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]],

    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]],

    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]],

    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]],

    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]],

    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]],

    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]],

    [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1],
     [1, 1, 1, 1, 0],
     [1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1]]
])


In [ ]:
N_data = N_data.reshape(10,25)
A_data = A_data.reshape(10,25)
R_data = R_data.reshape(10,25)

In [ ]:
dataset = []
labels = []
for k,i in enumerate([N_data,A_data,R_data]):
    dataset.append(i)
    for j in range(10):
        labels.append(k)
dataset = np.concatenate(dataset,axis = 0)
labels = np.array(labels)

In [ ]:
dataset.shape

(30, 25)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(dataset,labels,train_size = 0.7)

In [ ]:

batch_size = 21
input_size = 25
hidden_size = 10
output_size = 3
learning_rate = 0.01

w1 = np.random.randn(input_size, hidden_size).astype(np.float32)
w2 = np.random.randn(hidden_size, output_size).astype(np.float32)
w1_gpu = drv.mem_alloc(w1.nbytes)
w2_gpu = drv.mem_alloc(w2.nbytes)
drv.memcpy_htod(w1_gpu, w1)
drv.memcpy_htod(w2_gpu, w2)

In [ ]:
mod = SourceModule("""
__global__ void forward(float *input, float *w1, float *w2, float *z1, float *z2,
                       int input_size, int hidden_size, int output_size, int batch_size) {
    int batch_idx = blockIdx.x;
    int hidden_idx = threadIdx.x;

    if (hidden_idx < hidden_size) {
        float sum = 0.0f;
        for (int i = 0; i < input_size; i++) {
            sum += input[batch_idx * input_size + i] * w1[i * hidden_size + hidden_idx];
        }
        z1[batch_idx * hidden_size + hidden_idx] = fmaxf(0.0f, sum);
    }

    __syncthreads();

    int output_idx = threadIdx.x;
    if (output_idx < output_size) {
        float sum = 0.0f;
        for (int j = 0; j < hidden_size; j++) {
            sum += z1[batch_idx * hidden_size + j] * w2[j * output_size + output_idx];
        }
        z2[batch_idx * output_size + output_idx] = sum;
    }
}

__global__ void backprop(float *input, float *z1, float *z2, float *target, float *w1, float *w2,
                        int input_size, int hidden_size, int output_size, int batch_size, float learning_rate) {
    int batch_idx = blockIdx.x;
    int output_idx = threadIdx.x;

    if (output_idx < output_size) {
        float delta2 = z2[batch_idx * output_size + output_idx] - target[batch_idx * output_size + output_idx];
        for (int j = 0; j < hidden_size; j++) {
            atomicAdd(&w2[j * output_size + output_idx],
                     -learning_rate * delta2 * z1[batch_idx * hidden_size + j]);
        }
    }

    int hidden_idx = threadIdx.x;
    if (hidden_idx < hidden_size) {
        float delta1 = 0.0f;
        for (int k = 0; k < output_size; k++) {
            delta1 += (z2[batch_idx * output_size + k] - target[batch_idx * output_size + k])
                     * w2[hidden_idx * output_size + k];
        }
        delta1 *= (z1[batch_idx * hidden_size + hidden_idx] > 0.0f) ? 1.0f : 0.0f;

        for (int i = 0; i < input_size; i++) {
            atomicAdd(&w1[i * hidden_size + hidden_idx],
                     -learning_rate * delta1 * input[batch_idx * input_size + i]);
        }
    }
}
""")

forward_kernel = mod.get_function("forward")
backprop_kernel = mod.get_function("backprop")

In [ ]:
class ANN:
    def __init__(self, input_size=25, hidden_size=64, output_size=3, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        self.w1 = np.random.randn(input_size, hidden_size).astype(np.float32) * np.sqrt(2.0 / input_size)
        self.w2 = np.random.randn(hidden_size, output_size).astype(np.float32) * np.sqrt(2.0 / hidden_size)

        self.forward_kernel = mod.get_function("forward")
        self.backprop_kernel = mod.get_function("backprop")

        self.w1_gpu = gpuarray.to_gpu(self.w1)
        self.w2_gpu = gpuarray.to_gpu(self.w2)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def train_batch(self, batch_x, batch_y):
        batch_size = len(batch_x)

        inputs_gpu = gpuarray.to_gpu(batch_x.astype(np.float32))
        targets_gpu = gpuarray.to_gpu(batch_y.astype(np.float32))
        z1_gpu = gpuarray.zeros((batch_size, self.hidden_size), dtype=np.float32)
        z2_gpu = gpuarray.zeros((batch_size, self.output_size), dtype=np.float32)

        block_dim = (max(self.hidden_size, self.output_size), 1, 1)
        grid_dim = (batch_size, 1, 1)

        self.forward_kernel(inputs_gpu, self.w1_gpu, self.w2_gpu, z1_gpu, z2_gpu,
                          np.int32(self.input_size), np.int32(self.hidden_size),
                          np.int32(self.output_size), np.int32(batch_size),
                          block=block_dim, grid=grid_dim)

        outputs = self.softmax(z2_gpu.get())
        z2_gpu.gpudata.free()
        z2_gpu = gpuarray.to_gpu(outputs.astype(np.float32))

        self.backprop_kernel(inputs_gpu, z1_gpu, z2_gpu, targets_gpu,
                           self.w1_gpu, self.w2_gpu,
                           np.int32(self.input_size), np.int32(self.hidden_size),
                           np.int32(self.output_size), np.int32(batch_size),
                           np.float32(self.learning_rate),
                           block=block_dim, grid=grid_dim)

        inputs_gpu.gpudata.free()
        targets_gpu.gpudata.free()
        z1_gpu.gpudata.free()
        z2_gpu.gpudata.free()

    def predict(self, X):
        batch_size = len(X)

        inputs_gpu = gpuarray.to_gpu(X.astype(np.float32))
        z1_gpu = gpuarray.zeros((batch_size, self.hidden_size), dtype=np.float32)
        z2_gpu = gpuarray.zeros((batch_size, self.output_size), dtype=np.float32)

        block_dim = (max(self.hidden_size, self.output_size), 1, 1)
        grid_dim = (batch_size, 1, 1)

        self.forward_kernel(inputs_gpu, self.w1_gpu, self.w2_gpu, z1_gpu, z2_gpu,
                          np.int32(self.input_size), np.int32(self.hidden_size),
                          np.int32(self.output_size), np.int32(batch_size),
                          block=block_dim, grid=grid_dim)

        outputs = self.softmax(z2_gpu.get())

        inputs_gpu.gpudata.free()
        z1_gpu.gpudata.free()
        z2_gpu.gpudata.free()

        return outputs

def train_and_evaluate(X_train, y_train, X_test, y_test, epochs=100, batch_size=32):
    net = ANN(input_size=25, hidden_size=10, output_size=3)

    y_train_onehot = np.zeros((len(y_train), 3))
    y_train_onehot[np.arange(len(y_train)), y_train] = 1

    n_batches = len(X_train) // batch_size
    print("Training started...")

    for epoch in range(epochs):
        indices = np.random.permutation(len(X_train))
        X_train_shuffled = X_train[indices]
        y_train_shuffled = y_train_onehot[indices]

        for batch in range(n_batches):
            start_idx = batch * batch_size
            end_idx = start_idx + batch_size
            batch_x = X_train_shuffled[start_idx:end_idx]
            batch_y = y_train_shuffled[start_idx:end_idx]
            net.train_batch(batch_x, batch_y)

        if (epoch + 1) % 10 == 0:
            train_preds = np.argmax(net.predict(X_train), axis=1)
            train_acc = accuracy_score(y_train, train_preds)
            print(f"Epoch {epoch + 1}/{epochs}, Training Accuracy: {train_acc:.4f}")

    test_preds = np.argmax(net.predict(X_test), axis=1)
    test_acc = accuracy_score(y_test, test_preds)
    print("\nTest Results:")
    print(f"Test Accuracy: {test_acc:.4f}")

    return net

In [ ]:
model = train_and_evaluate(X_train, y_train, X_test, y_test, epochs=50, batch_size=21)

Training started...
Epoch 10/50, Training Accuracy: 0.9524
Epoch 20/50, Training Accuracy: 0.9524
Epoch 30/50, Training Accuracy: 1.0000
Epoch 40/50, Training Accuracy: 1.0000
Epoch 50/50, Training Accuracy: 1.0000

Test Results:
Test Accuracy: 0.8889


In [ ]:
mod = SourceModule("""
__global__ void conv2d(float *input, float *filters, float *output,
                      int batch_size, int height, int width, int channels,
                      int num_filters, int kernel_size, int output_height, int output_width) {
    int batch_idx = blockIdx.x;
    int filter_idx = blockIdx.y;
    int out_y = threadIdx.x;
    int out_x = threadIdx.y;
    if (out_y < output_height && out_x < output_width) {
        float sum = 0.0f;

        for (int c = 0; c < channels; c++) {
            for (int ky = 0; ky < kernel_size; ky++) {
                for (int kx = 0; kx < kernel_size; kx++) {
                    int in_y = out_y + ky;
                    int in_x = out_x + kx;

                    if (in_y < height && in_x < width) {
                        int input_idx = batch_idx * (height * width * channels) +
                                      c * (height * width) +
                                      in_y * width + in_x;
                        int filter_idx_full = filter_idx * (channels * kernel_size * kernel_size) +
                                            c * (kernel_size * kernel_size) +
                                           ky * kernel_size + kx;

                        sum += input[input_idx] * filters[filter_idx_full];
                    }
                }
            }
        }

        int output_idx = batch_idx * (num_filters * output_height * output_width) +
                        filter_idx * (output_height * output_width) +
                        out_y * output_width + out_x;
        output[output_idx] = fmaxf(0.0f, sum); // ReLU activation
    }
}
__global__ void max_pool2d(float *input, float *output,
                          int batch_size, int height, int width, int channels,
                          int pool_size, int output_height, int output_width) {
    int batch_idx = blockIdx.x;
    int channel = blockIdx.y;
    int out_y = threadIdx.x;
    int out_x = threadIdx.y;

    if (out_y < output_height && out_x < output_width) {
        float max_val = -1e10f;

        for (int py = 0; py < pool_size; py++) {
            for (int px = 0; px < pool_size; px++) {
                int in_y = out_y * pool_size + py;
                int in_x = out_x * pool_size + px;

                if (in_y < height && in_x < width) {
                    int input_idx = batch_idx * (channels * height * width) +
                                  channel * (height * width) +
                                  in_y * width + in_x;
                    max_val = fmaxf(max_val, input[input_idx]);
                }
            }
        }

        int output_idx = batch_idx * (channels * output_height * output_width) +
                        channel * (output_height * output_width) +
                        out_y * output_width + out_x;
        output[output_idx] = max_val;
        }
    }
__global__ void fc_forward(float *input, float *weights, float *output,
                          int batch_size, int input_size, int output_size) {
    int batch_idx = blockIdx.x;
    int neuron_idx = threadIdx.x;

    if (neuron_idx < output_size) {
        float sum = 0.0f;
        for (int i = 0; i < input_size; i++) {
            sum += input[batch_idx * input_size + i] * weights[i * output_size + neuron_idx];
        }
        output[batch_idx * output_size + neuron_idx] = sum;
    }
}
__global__ void conv2d_backward(float *d_output, float *filters, float *d_input,
                               int batch_size, int height, int width, int channels,
                               int num_filters, int kernel_size, int output_height, int output_width) {
    int batch_idx = blockIdx.x;
    int channel = blockIdx.y;
    int in_y = threadIdx.x;
    int in_x = threadIdx.y;

    if (in_y < height && in_x < width) {
        float sum = 0.0f;

        for (int f = 0; f < num_filters; f++) {
            for (int ky = 0; ky < kernel_size; ky++) {
                for (int kx = 0; kx < kernel_size; kx++) {
                    int out_y = in_y - ky;
                    int out_x = in_x - kx;

                    if (out_y >= 0 && out_y < output_height && out_x >= 0 && out_x < output_width) {
                        int d_output_idx = batch_idx * (num_filters * output_height * output_width) +
                                         f * (output_height * output_width) +
                                         out_y * output_width + out_x;
                        int filter_idx = f * (channels * kernel_size * kernel_size) +
                                       channel * (kernel_size * kernel_size) +
                                       ky * kernel_size + kx;

                        sum += d_output[d_output_idx] * filters[filter_idx];
                    }
                }
            }
        }

        int d_input_idx = batch_idx * (channels * height * width) +
                         channel * (height * width) +
                         in_y * width + in_x;
        d_input[d_input_idx] = sum;
    }
}
__global__ void conv2d_update_weights(float *input, float *d_output, float *d_weights,
                                    int batch_size, int height, int width, int channels,
                                    int num_filters, int kernel_size, int output_height, int output_width) {
    int filter_idx = blockIdx.x;
    int ky = threadIdx.x;
    int kx = threadIdx.y;
    int c = blockIdx.y;

    if (ky < kernel_size && kx < kernel_size) {
        float sum = 0.0f;

        for (int b = 0; b < batch_size; b++) {
            for (int out_y = 0; out_y < output_height; out_y++) {
                for (int out_x = 0; out_x < output_width; out_x++) {
                    int in_y = out_y + ky;
                    int in_x = out_x + kx;

                    if (in_y < height && in_x < width) {
                        int input_idx = b * (channels * height * width) +
                                      c * (height * width) +
                                      in_y * width + in_x;
                        int d_output_idx = b * (num_filters * output_height * output_width) +
                                         filter_idx * (output_height * output_width) +
                                         out_y * output_width + out_x;

                        sum += input[input_idx] * d_output[d_output_idx];
                    }
                }
            }

        }

        int weight_idx = filter_idx * (channels * kernel_size * kernel_size) +
                        c * (kernel_size * kernel_size) +
                        ky * kernel_size + kx;
        d_weights[weight_idx] = sum / batch_size;
    }
}
__global__ void max_pool_backward(float *input, float *d_output, float *d_input,
                                int batch_size, int height, int width, int channels,
                                int pool_size, int output_height, int output_width) {
    int batch_idx = blockIdx.x;
    int channel = blockIdx.y;
    int in_y = threadIdx.x;
    int in_x = threadIdx.y;

    if (in_y < height && in_x < width) {
        int out_y = in_y / pool_size;
        int out_x = in_x / pool_size;

        if (out_y < output_height && out_x < output_width) {
            int input_idx = batch_idx * (channels * height * width) +
                           channel * (height * width) +
                           in_y * width + in_x;
            int output_idx = batch_idx * (channels * output_height * output_width) +
                           channel * (output_height * output_width) +
                           out_y * output_width + out_x;
           float input_val = input[input_idx];
            float max_val = -1e10f;
            int max_idx_y = -1, max_idx_x = -1;

            for (int py = 0; py < pool_size; py++) {
                for (int px = 0; px < pool_size; px++) {
                    int curr_y = out_y * pool_size + py;
                    int curr_x = out_x * pool_size + px;

                    if (curr_y < height && curr_x < width) {
                        int curr_idx = batch_idx * (channels * height * width) +
                                     channel * (height * width) +
                                     curr_y * width + curr_x;
                        float curr_val = input[curr_idx];

                        if (curr_val > max_val) {
                            max_val = curr_val;
                            max_idx_y = curr_y;
                            max_idx_x = curr_x;
                        }
                    }
                }
            }

            if (in_y == max_idx_y && in_x == max_idx_x) {
                d_input[input_idx] = d_output[output_idx];
            } else {
                d_input[input_idx] = 0.0f;
            }
        }
    }
}
__global__ void fc_backward(float *d_output, float *weights, float *d_input,
                           int batch_size, int input_size, int output_size) {
    int batch_idx = blockIdx.x;
    int input_idx = threadIdx.x;

    if (input_idx < input_size) {
        float sum = 0.0f;
        for (int j = 0; j < output_size; j++) {
            sum += d_output[batch_idx * output_size + j] * weights[input_idx * output_size + j];
        }
        d_input[batch_idx * input_size + input_idx] = sum;
    }
}

__global__ void fc_update_weights(float *input, float *d_output, float *d_weights,
                                int batch_size, int input_size, int output_size) {
    int in_idx = blockIdx.x;
    int out_idx = threadIdx.x;

    if (in_idx < input_size && out_idx < output_size) {
        float sum = 0.0f;
        for (int b = 0; b < batch_size; b++) {
            sum += input[b * input_size + in_idx] * d_output[b * output_size + out_idx];
        }
        d_weights[in_idx * output_size + out_idx] = sum / batch_size;
    }
}
__global__ void relu(float *x, int n) {
            int idx = blockIdx.x * blockDim.x + threadIdx.x;
            if (idx < n) {
                x[idx] = max(0.0, x[idx]);
            }
    }
__global__ void softmax(float *x, int batch_size, int num_classes) {
            int batch_idx = blockIdx.x;
            int class_idx = threadIdx.x;

            int idx = batch_idx * num_classes + class_idx;

            float max_val = -1e9;
            for (int j = 0; j < num_classes; j++) {
                max_val = fmaxf(max_val, x[batch_idx * num_classes + j]);
            }

            float sum_exp = 0.0;
            for (int j = 0; j < num_classes; j++) {
                int current_idx = batch_idx * num_classes + j;
                x[current_idx] = expf(x[current_idx] - max_val);
                sum_exp += x[current_idx];
            }

            x[idx] = x[idx] / sum_exp;
        }
""")

In [ ]:
class CNN:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        self.conv1_filters = 32
        self.conv1_size = 3
        self.pool_size = 2
        self.fc1_size = 10
        self.num_classes = 3

        self.input_size = 5
        self.conv1_output_size = self.input_size - self.conv1_size + 1
        self.pool1_output_size = self.conv1_output_size // self.pool_size

        conv1_weights = np.random.randn(self.conv1_filters, 1,
                                      self.conv1_size, self.conv1_size).astype(np.float32) * 0.1
        self.conv1_weights_gpu = gpuarray.to_gpu(conv1_weights)

        fc1_input_size = self.conv1_filters * self.pool1_output_size * self.pool1_output_size
        fc1_weights = np.random.randn(fc1_input_size, self.fc1_size).astype(np.float32) * 0.1
        self.fc1_weights_gpu = gpuarray.to_gpu(fc1_weights)

        fc2_weights = np.random.randn(self.fc1_size, self.num_classes).astype(np.float32) * 0.1
        self.fc2_weights_gpu = gpuarray.to_gpu(fc2_weights)

        self.max_pool = mod.get_function("max_pool2d")
        self.fc_forward = mod.get_function("fc_forward")
        self.conv_forward = mod.get_function("conv2d")
        self.conv_backward = mod.get_function("conv2d_backward")
        self.conv_update_weights = mod.get_function("conv2d_update_weights")
        self.pool_backward = mod.get_function("max_pool_backward")
        self.fc_backward = mod.get_function("fc_backward")
        self.fc_update_weights = mod.get_function("fc_update_weights")
        self.relu = mod.get_function("relu")
        self.softmax = mod.get_function("softmax")
        self.train_losses = []
        self.train_accuracies = []
        self.eval_accuracies = []

    def backward(self, x_batch, y_batch, output):
        batch_size = len(x_batch)

        d_output = output.copy()
        d_output[range(batch_size), y_batch] -= 1
        d_output /= batch_size

        d_fc2_output_gpu = gpuarray.to_gpu(d_output)
        d_fc1_output = gpuarray.zeros((batch_size, self.fc1_size), dtype=np.float32)

        block_dim = (self.fc1_size, 1, 1)
        grid_dim = (batch_size, 1, 1)

        self.fc_backward(d_fc2_output_gpu, self.fc2_weights_gpu, d_fc1_output,
                        np.int32(batch_size), np.int32(self.fc1_size), np.int32(self.num_classes),
                        block=block_dim, grid=grid_dim)

        d_fc2_weights = gpuarray.zeros_like(self.fc2_weights_gpu)

        block_dim = (self.num_classes, 1, 1)
        grid_dim = (self.fc1_size, 1, 1)

        self.fc_update_weights(d_fc1_output, d_fc2_output_gpu, d_fc2_weights,
                             np.int32(batch_size), np.int32(self.fc1_size), np.int32(self.num_classes),
                             block=block_dim, grid=grid_dim)

        d_pool1_output = gpuarray.zeros((batch_size, self.conv1_filters,
                                       self.pool1_output_size, self.pool1_output_size),
                                      dtype=np.float32)

        flattened_size = self.conv1_filters * self.pool1_output_size * self.pool1_output_size

        block_dim = (flattened_size, 1, 1)
        grid_dim = (batch_size, 1, 1)

        self.fc_backward(d_fc1_output, self.fc1_weights_gpu, d_pool1_output,
                        np.int32(batch_size), np.int32(flattened_size), np.int32(self.fc1_size),
                        block=block_dim, grid=grid_dim)
        d_fc1_weights = gpuarray.zeros_like(self.fc1_weights_gpu)

        block_dim = (self.fc1_size, 1, 1)
        grid_dim = (flattened_size, 1, 1)

        self.fc_update_weights(d_pool1_output, d_fc1_output, d_fc1_weights,
                             np.int32(batch_size), np.int32(flattened_size), np.int32(self.fc1_size),
                             block=block_dim, grid=grid_dim)

        d_conv1_output = gpuarray.zeros((batch_size, self.conv1_filters,
                                       self.conv1_output_size, self.conv1_output_size),
                                      dtype=np.float32)

        block_dim = (self.conv1_output_size, self.conv1_output_size, 1)
        grid_dim = (batch_size, self.conv1_filters, 1)

        self.pool_backward(d_conv1_output, d_pool1_output, d_conv1_output,
                          np.int32(batch_size), np.int32(self.conv1_output_size),
                          np.int32(self.conv1_output_size), np.int32(self.conv1_filters),
                          np.int32(self.pool_size), np.int32(self.pool1_output_size),
                          np.int32(self.pool1_output_size),
                          block=block_dim, grid=grid_dim)

        d_input = gpuarray.zeros((batch_size, 1, 5, 5), dtype=np.float32)

        block_dim = (5, 5, 1)
        grid_dim = (batch_size, 1, 1)

        self.conv_backward(d_conv1_output, self.conv1_weights_gpu, d_input,
                          np.int32(batch_size), np.int32(5), np.int32(5), np.int32(1),
                          np.int32(self.conv1_filters), np.int32(self.conv1_size),
                          np.int32(self.conv1_output_size), np.int32(self.conv1_output_size),
                          block=block_dim, grid=grid_dim)

        d_conv1_weights = gpuarray.zeros_like(self.conv1_weights_gpu)

        block_dim = (self.conv1_size, self.conv1_size, 1)
        grid_dim = (self.conv1_filters, 1, 1)

        self.conv_update_weights(d_input, d_conv1_output, d_conv1_weights,
                               np.int32(batch_size), np.int32(5), np.int32(5), np.int32(1),
                               np.int32(self.conv1_filters), np.int32(self.conv1_size),
                               np.int32(self.conv1_output_size), np.int32(self.conv1_output_size),
                               block=block_dim, grid=grid_dim)

        self.conv1_weights_gpu -= self.learning_rate * d_conv1_weights
        self.fc1_weights_gpu -= self.learning_rate * d_fc1_weights
        self.fc2_weights_gpu -= self.learning_rate * d_fc2_weights

        d_fc2_output_gpu.gpudata.free()
        d_fc1_output.gpudata.free()
        d_pool1_output.gpudata.free()
        d_conv1_output.gpudata.free()
        d_input.gpudata.free()
        d_conv1_weights.gpudata.free()
        d_fc1_weights.gpudata.free()
        d_fc2_weights.gpudata.free()


    def compute_loss(self, output, y_batch):
        """Compute categorical cross-entropy loss."""
        batch_size = len(y_batch)
        output_cpu = output.get()
        log_likelihood = -np.log(output_cpu[range(batch_size), y_batch])
        loss = np.sum(log_likelihood) / batch_size
        return loss

    def compute_accuracy(self, output, y_batch):
        """Compute classification accuracy."""
        predictions = np.argmax(output.get(), axis=1)
        return np.mean(predictions == y_batch)

    def forward(self, x_batch):
        """Forward pass returning output probabilities."""
        batch_size = len(x_batch)

        input_gpu = gpuarray.to_gpu(x_batch.astype(np.float32))

        # Convolution layer
        conv1_output = gpuarray.zeros((batch_size, self.conv1_filters,
                                     self.conv1_output_size, self.conv1_output_size),
                                    dtype=np.float32)

        block_dim = (self.conv1_output_size, self.conv1_output_size, 1)
        grid_dim = (batch_size, self.conv1_filters, 1)

        self.conv_forward(input_gpu, self.conv1_weights_gpu, conv1_output,
                         np.int32(batch_size), np.int32(5), np.int32(5), np.int32(1),
                         np.int32(self.conv1_filters), np.int32(self.conv1_size),
                         np.int32(self.conv1_output_size), np.int32(self.conv1_output_size),
                         block=block_dim, grid=grid_dim)

        # Max pooling layer
        pool1_output = gpuarray.zeros((batch_size, self.conv1_filters,
                                     self.pool1_output_size, self.pool1_output_size),
                                    dtype=np.float32)

        block_dim = (self.pool1_output_size, self.pool1_output_size, 1)
        grid_dim = (batch_size, self.conv1_filters, 1)

        self.max_pool(conv1_output, pool1_output,
                     np.int32(batch_size), np.int32(self.conv1_output_size),
                     np.int32(self.conv1_output_size), np.int32(self.conv1_filters),
                     np.int32(self.pool_size), np.int32(self.pool1_output_size),
                     np.int32(self.pool1_output_size),
                     block=block_dim, grid=grid_dim)

        # Flatten pooling output
        flattened = pool1_output.reshape(batch_size, -1)

        # First fully connected layer
        fc1_output = gpuarray.zeros((batch_size, self.fc1_size), dtype=np.float32)

        block_dim = (self.fc1_size, 1, 1)
        grid_dim = (batch_size, 1, 1)

        self.fc_forward(flattened, self.fc1_weights_gpu, fc1_output,
                       np.int32(batch_size), np.int32(flattened.shape[1]),
                       np.int32(self.fc1_size),
                       block=block_dim, grid=grid_dim)

        # ReLU activation
        self.relu(fc1_output, np.int32(fc1_output.size),
                 block=(256, 1, 1), grid=((fc1_output.size + 255) // 256, 1, 1))

        # Second fully connected layer (output layer)
        output = gpuarray.zeros((batch_size, self.num_classes), dtype=np.float32)

        block_dim = (self.num_classes, 1, 1)
        grid_dim = (batch_size, 1, 1)

        self.fc_forward(fc1_output, self.fc2_weights_gpu, output,
                       np.int32(batch_size), np.int32(self.fc1_size),
                       np.int32(self.num_classes),
                       block=block_dim, grid=grid_dim)

        # Softmax activation
        self.softmax(output, np.int32(batch_size), np.int32(self.num_classes),
                    block=(self.num_classes, 1, 1), grid=(batch_size, 1, 1))

        return output, conv1_output, pool1_output, fc1_output

    def train(self, X_train, y_train, X_val=None, y_val=None, epochs=10, batch_size=32):
        """Train the CNN using mini-batch gradient descent."""
        n_samples = len(X_train)
        n_batches = (n_samples + batch_size - 1) // batch_size

        for epoch in range(epochs):
            indices = np.random.permutation(n_samples)
            X_train = X_train[indices]
            y_train = y_train[indices]

            epoch_loss = 0
            epoch_accuracy = 0

            for batch in range(n_batches):
                start_idx = batch * batch_size
                end_idx = min((batch + 1) * batch_size, n_samples)

                x_batch = X_train[start_idx:end_idx]
                y_batch = y_train[start_idx:end_idx]

                output, conv1_output, pool1_output, fc1_output = self.forward(x_batch)

                batch_loss = self.compute_loss(output, y_batch)
                batch_accuracy = self.compute_accuracy(output, y_batch)

                epoch_loss += batch_loss * len(x_batch)
                epoch_accuracy += batch_accuracy * len(x_batch)

                self.backward(x_batch, y_batch, output.get())

                conv1_output.gpudata.free()
                pool1_output.gpudata.free()
                fc1_output.gpudata.free()
                output.gpudata.free()

            epoch_loss /= n_samples
            epoch_accuracy /= n_samples

            self.train_losses.append(epoch_loss)
            self.train_accuracies.append(epoch_accuracy)
            print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.4f}")

    def evaluate(self, X_test, y_test, batch_size=32):
        """Evaluate the CNN on test data."""
        n_samples = len(X_test)
        n_batches = (n_samples + batch_size - 1) // batch_size
        total_accuracy = 0

        for batch in range(n_batches):
            start_idx = batch * batch_size
            end_idx = min((batch + 1) * batch_size, n_samples)

            x_batch = X_test[start_idx:end_idx]
            y_batch = y_test[start_idx:end_idx]

            output, conv1_output, pool1_output, fc1_output = self.forward(x_batch)

            batch_accuracy = self.compute_accuracy(output, y_batch)
            total_accuracy += batch_accuracy * len(x_batch)

            conv1_output.gpudata.free()
            pool1_output.gpudata.free()
            fc1_output.gpudata.free()
            output.gpudata.free()

        return total_accuracy / n_samples

    def predict(self, X, batch_size=32):
        """Generate predictions for input data."""
        n_samples = len(X)
        n_batches = (n_samples + batch_size - 1) // batch_size
        predictions = np.zeros(n_samples, dtype=np.int32)

        for batch in range(n_batches):
            start_idx = batch * batch_size
            end_idx = min((batch + 1) * batch_size, n_samples)

            x_batch = X[start_idx:end_idx]

            output, conv1_output, pool1_output, fc1_output = self.forward(x_batch)

            batch_predictions = np.argmax(output.get(), axis=1)
            predictions[start_idx:end_idx] = batch_predictions

            # Free GPU memory
            conv1_output.gpudata.free()
            pool1_output.gpudata.free()
            fc1_output.gpudata.free()
            output.gpudata.free()

        return predictions

In [ ]:
cnn = CNN(learning_rate=1e-4)

cnn.train(X_train, y_train, X_test, y_test, epochs=10, batch_size=21)
test_accuracy = cnn.evaluate(X_test, y_test)
print(f"Test Accuracy : {test_accuracy}")

Epoch 1/10 - Loss: 1.0989 - Accuracy: 0.2857
Epoch 2/10 - Loss: 1.0989 - Accuracy: 0.2857
Epoch 3/10 - Loss: 1.0989 - Accuracy: 0.2857
Epoch 4/10 - Loss: 1.0989 - Accuracy: 0.2857
Epoch 5/10 - Loss: 1.0989 - Accuracy: 0.2857
Epoch 6/10 - Loss: 1.0989 - Accuracy: 0.2857
Epoch 7/10 - Loss: 1.0989 - Accuracy: 0.2857
Epoch 8/10 - Loss: 1.0989 - Accuracy: 0.2857
Epoch 9/10 - Loss: 1.0989 - Accuracy: 0.2857
Epoch 10/10 - Loss: 1.0989 - Accuracy: 0.2857
Test Accuracy : 0.5555555555555556


/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()
